In [1]:
import pandas as pd

In [2]:
dv=pd.read_csv('delivery_raw.csv', sep='\t')

In [3]:
dv.tail()

,market_id,created_at,actual_delivery_time,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,min_item_price,max_item_price,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration
197423,1.0,2015-02-17 00:19:41,2015-02-17 01:24:48,2956,fast,4.0,3,1389,3,345,649,17.0,17.0,23.0,251,331.0
197424,1.0,2015-02-13 00:01:59,2015-02-13 00:58:22,2956,fast,4.0,6,3010,4,405,825,12.0,11.0,14.0,251,915.0
197425,1.0,2015-01-24 04:46:08,2015-01-24 05:36:16,2956,fast,4.0,5,1836,3,300,399,39.0,41.0,40.0,251,795.0
197426,1.0,2015-02-01 18:18:15,2015-02-01 19:23:22,3630,sandwich,1.0,1,1175,1,535,535,7.0,7.0,12.0,446,384.0
197427,1.0,2015-02-08 19:24:33,2015-02-08 20:01:41,3630,sandwich,1.0,4,2605,4,425,750,20.0,20.0,23.0,446,134.0


In [5]:
dv.isna().sum()

market_id                                         987
created_at                                          0
actual_delivery_time                                7
store_id                                            0
store_primary_category                           4760
order_protocol                                    995
total_items                                         0
subtotal                                            0
num_distinct_items                                  0
min_item_price                                      0
max_item_price                                      0
total_onshift                                   16262
total_busy                                      16262
total_outstanding_orders                        16262
estimated_order_place_duration                      0
estimated_store_to_consumer_driving_duration      526
dtype: int64

약 20만 개의 데이터로 수가 굉장히 많기 때문에 1.6만개 가량의 결측치가 포함된 데이터는 삭제하였다.

In [6]:
dv=dv.dropna('index');

C:\Users\gka06\AppData\Local\Temp/ipykernel_8872/363405307.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only
  dv=dv.dropna('index');


#### 주문시각과 도착시각을 이용해 경과시간을 추출하였다.

In [8]:
dv['created_at']=pd.to_datetime(dv['created_at'])
dv['actual_delivery_time']=pd.to_datetime(dv['actual_delivery_time'])

In [17]:
dv['time_taken']=(dv['actual_delivery_time']-dv['created_at']).dt.total_seconds()
dv.drop(['actual_delivery_time', 'created_at'], axis=1, inplace=True)
dv['time_taken']

0         3779.0
1         4024.0
8         1586.0
14        2273.0
15        2988.0
           ...  
197423    3907.0
197424    3383.0
197425    3008.0
197426    3907.0
197427    2228.0
Name: time_taken, Length: 175777, dtype: float64

In [16]:
dv.corr()['time_taken']

market_id                                      -0.037958
store_id                                       -0.005618
order_protocol                                 -0.047027
total_items                                     0.077331
subtotal                                        0.143549
num_distinct_items                              0.105881
min_item_price                                  0.009013
max_item_price                                  0.089278
total_onshift                                   0.047321
total_busy                                      0.060864
total_outstanding_orders                        0.122026
estimated_order_place_duration                  0.068715
estimated_store_to_consumer_driving_duration    0.158361
time_taken                                      1.000000
Name: time_taken, dtype: float64

In [ ]:
#### 경과시간과 관계가 적은 store_id, min_item_price를 제거하고

In [18]:
dv.columns

Index(['market_id', 'store_id', 'store_primary_category', 'order_protocol',
       'total_items', 'subtotal', 'num_distinct_items', 'min_item_price',
       'max_item_price', 'total_onshift', 'total_busy',
       'total_outstanding_orders', 'estimated_order_place_duration',
       'estimated_store_to_consumer_driving_duration', 'time_taken'],
      dtype='object')

In [16]:
dv.dtypes

market_id                                              float64
created_at                                      datetime64[ns]
actual_delivery_time                            datetime64[ns]
store_id                                                 int64
store_primary_category                                  object
order_protocol                                         float64
total_items                                              int64
subtotal                                                 int64
num_distinct_items                                       int64
min_item_price                                           int64
max_item_price                                           int64
total_onshift                                          float64
total_busy                                             float64
total_outstanding_orders                               float64
estimated_order_place_duration                           int64
estimated_store_to_consumer_driving_duration           

#### 이 열 중에서 학습에 필요치 않은 열들(경과시간과 관련성이 적음)을 제거하였다.(min_item_price, max_item_price)

In [19]:
dv.drop(['min_item_price', 'max_item_price'], axis=1, inplace=True)
dv

,market_id,store_id,store_primary_category,order_protocol,total_items,subtotal,num_distinct_items,total_onshift,total_busy,total_outstanding_orders,estimated_order_place_duration,estimated_store_to_consumer_driving_duration,time_taken
0,1.0,1845,american,1.0,4,3441,4,33.0,14.0,21.0,446,861.0,3779.0
1,2.0,5477,mexican,2.0,1,1900,1,1.0,2.0,2.0,446,690.0,4024.0
8,2.0,5477,indian,3.0,4,4771,3,8.0,6.0,18.0,446,289.0,1586.0
14,1.0,2841,italian,1.0,1,1525,1,5.0,6.0,8.0,446,795.0,2273.0
15,1.0,2841,italian,1.0,2,3620,2,5.0,5.0,7.0,446,205.0,2988.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
197423,1.0,2956,fast,4.0,3,1389,3,17.0,17.0,23.0,251,331.0,3907.0
197424,1.0,2956,fast,4.0,6,3010,4,12.0,11.0,14.0,251,915.0,3383.0
197425,1.0,2956,fast,4.0,5,1836,3,39.0,41.0,40.0,251,795.0,3008.0
197426,1.0,3630,sandwich,1.0,1,1175,1,7.0,7.0,12.0,446,384.0,3907.0
